### Content:
- Use SpaCy NLP to find companys
- Match companys in news with stock companies (using regex)
- Exclude matches in headers afterwards (using regex)
- Apply NLP on one Reuters and one Bloomberg article as show cases
- Fix matchings for 3M Company
- Generate cooccurrences (value represents number of articles in which two companies occur together)

#### TODO:
- Read through some articles to find a show case how it relates to stock prices

In [1]:
import os
import re
import glob
from datetime import datetime
import sys
sys.path.append("..") # Adds higher directory to python modules path for importing from src dir

import pandas as pd
import numpy as np
import tqdm
import matplotlib
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load(disable=['parser', 'tagger'])

%matplotlib inline
%load_ext autotime
%load_ext autoreload
%autoreload 2

c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [2]:
HOME = ".."
DATA_DIR = "data"
REUTERS = os.path.join(HOME, DATA_DIR, "preprocessed", "news_reuters.csv")
BLOOMBERG = os.path.join(HOME, DATA_DIR, "preprocessed", "news_bloomberg.csv")
NEWS = os.path.join(HOME, DATA_DIR, "preprocessed", "news.csv")
# Columns: 'date', 'filename', 'content'

time: 19.1 ms


In [3]:
news = pd.read_csv(NEWS, index_col=0)

# 8650 duplicated filenames, 786 duplicated contents
# d = news.filename.duplicated()
# print(news.loc[1716].iloc[1].content) # 1201 letters
# print(news.loc[1641].iloc[1].content) # 541 letters

time: 25.1 s


In [80]:
def get_old_idx(article):
    if article.reuters:
        return f'r{article.name}'
    return f'b{article.name}'

# [get_old_idx(idx, article) for idx, article in news.iterrows()];
news['old_idx'] = news.apply(get_old_idx, axis=1)

time: 16.2 s


In [86]:
news.reset_index(drop=True, inplace=True)
news = news[['old_idx', 'date', 'filename', 'content']]

new_to_old_idx = news.old_idx.to_dict()
old_to_new_idx = {v: k for k, v in new_to_old_idx.items()}

time: 24.4 ms


In [102]:
news.to_csv('news-v2.csv')

time: 48.2 s


## Sanity Check for Entities

In [4]:
ENTITIES = os.path.join("..", "data", "preprocessed", "entities.csv")
NEWS = os.path.join("..", "data", "preprocessed", "news-v2.csv")

time: 124 ms


In [5]:
entities = pd.read_csv(ENTITIES, index_col=0)
news = pd.read_csv(NEWS, index_col=0)

c:\users\thomas\hpi\ma\venv\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 1min 16s


In [6]:
entities.columns

Index(['article_id', 'match_text', 'start_idx', 'end_idx', 'label'], dtype='object')

time: 391 ms


In [7]:
news.columns

Index(['old_idx', 'date', 'filename', 'content'], dtype='object')

time: 180 ms
